In [2]:
import re 
import pandas as pd  
import numpy as np
import ast

## The task here is to clean a dataset of thousands of names associated with ID numbers... 
***-- Many of these ID numbers are associated with more than one name;***

***-- Many of those duplicate names are the same name spelt differently;***

***-- some of them are different people erroneously associated with the same ID number.***

## Read-in Dataset

In [3]:
# Read in CSV data and parse-normalize dates 
parse_dates = ['Date of Birth']
names = pd.read_csv('test_data/duplicates_apps.csv', encoding="utf8",parse_dates=parse_dates)

In [4]:
# delete obsolete unnamed column
names.columns[0]
del names[names.columns[0]]

In [5]:
# Find and delete duplicate= rows accross all columns:
#duplicateRowsDF = names[names.duplicated()]
#print("Duplicate Rows except first occurrence based on all columns are :")
#print(duplicateRowsDF
names_set = names.drop_duplicates()
print("Number of duplicate rows deleted:", len(names[names.duplicated()]))

Number of duplicate rows deleted: 3


In [6]:
# Input index column for referencing changes to origin dataframe
Names_set=names_set.copy()
Names_set['index'] = names_set.index

In [7]:
# convert NaN values to empty strings and lowercase all name values
Names_set['Employee Name']=Names_set['Employee Name'].replace(np.nan, '', regex=True)
Names_set['Family Name']=Names_set['Family Name'].replace(np.nan, '', regex=True)
Names_set['Employee Name']=Names_set['Employee Name'].str.lower()
Names_set['Family Name']=Names_set['Family Name'].str.lower()

In [8]:
# Strip each values of leading or trailing whitespace and remove extra whitespaces
Names_set['Employee Name']=Names_set['Employee Name'].str.strip()
Names_set['Family Name']=Names_set['Family Name'].str.strip()

Names_set['Employee Name'] = Names_set['Employee Name'].apply(lambda x: re.sub(r' +', ' ', x))
Names_set['Family Name'] = Names_set['Family Name'].apply(lambda x: re.sub(r' +', ' ', x))

In [9]:
FullNames = Names_set.loc[:, ['Employee Name','Family Name']]

In [10]:
# Prep text by converting NaN values to empty strings
import numpy as np
#EmployeeNames = EmployeeNames.replace(np.nan, '', regex=True)
#FamilyNames = FamilyNames.replace(np.nan, '', regex=True)
FullNames = FullNames.replace(np.nan, '', regex=True)

In [11]:
FullNames['index'] = FullNames.index # Set Index for new dataframe
FullNames["FullName"] = FullNames["Employee Name"] + ' ' + FullNames["Family Name"] # Set new column of joined name parts
FullNames["FullName"]=FullNames["FullName"].str.replace('.','')
FullNames["FullName"]=FullNames["FullName"].str.strip()
#EmployeeNames = Names_set.loc[:, ['Employee Name']]
#FamilyNames = Names_set.loc[:, ['Family Name']]
#EmployeeNames['index'] = EmployeeNames.index
#FamilyNames['index'] = FamilyNames.index

## N-Grams
n-grams: sequences of N contiguous items, in this case characters.

The terms in TF-IDF can be n-grams instead of words!

In [12]:
# This function cleans a string then generates all n-grams in the string
def ngrams(string, n=3):
    string = re.sub(r'[,-./]',r' ', string) # Remove simple punctuation, could always add more here...
    #string = re.sub(r'[,-./]|\sYOURSTRING',r'', string) #Replace YOURSTRING with any letters that may cause bias and need to be replaced.
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

## TF-IDF
Generates features from text by means of multiplying the frequency of a term in a document (TF)
by the importance (IDF) of that term throughout the corpus. 
IDF weights less important words down and words that don’t occur frequently up.

IDF(t) = log_e(Total number of documents / Number of documents with term t in it).

TF-IDF is used to transform documents into numeric vectors

In [13]:
# Here is the code to generate a matrix of TF-IDF values for each n-gram
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

from sklearn.feature_extraction.text import TfidfVectorizer

FullNamesList = list(set(list(FullNames['FullName'])))
#EmployeeNamesList = list(EmployeeNames['Employee Name'])
#EmployeeNamesList=list(set(EmployeeNamesList))
#FamilyNamesList = list(FamilyNames['Family Name'])
#FamilyNamesList=list(set(FamilyNamesList))

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams) #analyzer{‘word’, ‘char’, ‘char_wb’} or callable, default=’word’ so we feature should be made of word or character n-grams
tf_idf_matrix = vectorizer.fit_transform(FullNamesList)
#tf_idf_matrix = vectorizer.fit_transform(EmployeeNamesList)
#tf_idf_matrix2 = vectorizer.fit_transform(FamilyNamesList)
print(tf_idf_matrix.shape)

(53, 329)


## Alternative option for Cosine similarity provided by 
https://github.com/ing-bank/sparse_dot_topn

https://bergvca.github.io/2017/10/14/super-fast-string-matching.html

In [14]:
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

# Stores only the top N highest matches in each row, and only the similarities above an (optional) threshold.
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

**The following code runs the optimized cosine similarity function.**

**It only stores the top n most similar items, and only items with a similarity above n.nn:**

In [15]:
import time
t1 = time.time()
FullNamesMatches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 1000, 0.30)
#EmployeeNamesMatches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 1000, 0.20)
#FamilyNamesMatches = awesome_cossim_top(tf_idf_matrix2, tf_idf_matrix2.transpose(), 1000, 0.20)

#matches = cosine_similarity(tf_idf_matrix, tf_idf_matrix.transpose())
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 0.000997781753540039


**Unpack the sparse matrix with added option to look at only the first n values:**

In [16]:
def get_matches_df(sparse_matrix, name_vector, top=1000):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similarity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similarity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similarity})
        

**View matches in dataframe matches_df**

In [17]:
FullNamesMatchesDF = get_matches_df(FullNamesMatches, FullNamesList, top=len(FullNamesList)) # n in 'top=n' can only be within bounds of index on dataset
FullNamesMatchesDF = FullNamesMatchesDF[FullNamesMatchesDF['similarity'] < 0.99999] # Remove all exact matches

#EmployeeNamesMatchesDF = get_matches_df(EmployeeNamesMatches, EmployeeNamesList, top=len(EmployeeNamesList)) # n in 'top=n' can only be within bounds of index on dataset
#EmployeeNamesMatchesDF = EmployeeNamesMatchesDF[EmployeeNamesMatchesDF['similarity'] < 0.99999] # Remove all exact matches
#FamilyNamesMatchesDF = get_matches_df(FamilyNamesMatches, FamilyNamesList, top=len(FamilyNamesList)) # n in 'top=n' can only be within bounds of index on dataset
#FamilyNamesMatchesDF = FamilyNamesMatchesDF[FamilyNamesMatchesDF['similarity'] < 0.99999] # Remove all exact matches
#matches_df.sample(5) #will fail here if less results are found that samples requested 

In [19]:
len(list(FullNamesMatchesDF['right_side']))

30

In [20]:
#Show full dataframe
pd.set_option('max_rows', None)
pd.set_option('max_colwidth', None)

In [21]:
#FullNamesMatchesDF.sort_values(['similarity'], ascending=False)

**Remove duplicated entities criss-crossing both columns**
**by converting to frozenset and using pd.DataFrame.duplicated:**


In [22]:
final_matches_df = FullNamesMatchesDF[~FullNamesMatchesDF[['left_side', 'right_side']].apply(frozenset, axis=1).duplicated()]
#final_matches_df.sort_values(['similarity'], ascending=False)

**Use pivot to append duplicate rows in DataFrame to right hand side as new columns then do the same from the left hand side. Append each result as a dictionary to a list and then combine the two dictionaries to create a dictionary of similar values**

In [23]:
# Instantiate list to hold dictionaries: 
results=[]

In [24]:
#append duplicate rows in DataFrame to right hand side as new columns
#final_matches_df = final_matches_df.loc[:, ['left_side','right_side']]
result = (final_matches_df.assign(count=final_matches_df.groupby("left_side").cumcount())
            .pivot(index='left_side', columns='count'))

result.columns = ["_".join(str(x) for x in i) for i in result.columns]
#result.info()
result['left_side'] = result.index 
result = result.replace(np.nan, '', regex=True)

In [25]:
numbRightsideCols = len([c for c in result.columns if c.count('right_side')>0])
for i in range(numbRightsideCols):
    col1 = 'right_side_'+str(i)
    similarity_col = 'similarity_'+str(i)
    result[col1] = result[[col1,similarity_col]].apply(lambda row: tuple(row.values.astype(str)), axis=1)
    del result[similarity_col] # Delete similarity column

In [26]:
result2 = (final_matches_df.assign(count=final_matches_df.groupby("right_side").cumcount())
            .pivot(index='right_side', columns='count'))

result2.columns = ["_".join(str(x) for x in i) for i in result2.columns]
#result.info()
result2['right_side'] = result2.index 
result2 = result2.replace(np.nan, '', regex=True)

In [27]:
result2.columns

Index(['left_side_0', 'left_side_1', 'similarity_0', 'similarity_1',
       'right_side'],
      dtype='object')

In [28]:
numbLeftsideCols = len([c for c in result2.columns if c.count('left_side')>0])
for i in range(numbLeftsideCols):
    col1 = 'left_side_'+str(i)
    similarity_col = 'similarity_'+str(i)
    result2[col1] = result2[[col1,similarity_col]].apply(lambda row: tuple(row.values.astype(str)), axis=1)
    #result2[col1] = result2[col1].replace("('', '')", '', regex=False)
    del result2[similarity_col] # Delete similarity column

In [29]:
#result.set_index('left_side').to_dict()
left_result = result.set_index('left_side').T.to_dict('list')

In [30]:
for k,v in left_result.items():
    left_result[k] = [n for n in v if n != ('', '')]
results.append(left_result)

In [31]:
right_result = result2.set_index('right_side').T.to_dict('list')
for k,v in right_result.items():
    right_result[k] = [n for n in v if n != ('', '')]
results.append(right_result)

In [32]:
def merge_dictionary_list(dict_list):
  return {
    k: [d.get(k) for d in dict_list if k in d] # explanation A
    for k in set().union(*dict_list) # explanation B
  }

In [33]:
complete_results = merge_dictionary_list(results)

## Store All Associationed Names in Dictionary (term_similarity_table):

In [34]:
import itertools
term_similarity_table={}
for k,v in complete_results.items():
    #flattened_list = [i for sublist in v for i in sublist]
    #complete_results[k]=flattened_list
    merged = list(set(list(itertools.chain.from_iterable(v))))
    term_similarity_table[k]=merged

In [35]:
# Find and delete duplicate rows accross all columns after lowercasing and stripping the content:
Names_set = Names_set.drop_duplicates()
print("Number of duplicate rows deleted:", len(Names_set[Names_set.duplicated()]))

# Select all duplicate rows based on multiple column names in list
#duplicateAPPSid = names_set[names_set.duplicated(['APPS ID'])]
duplicateAPPSid = pd.concat(row for _, row in Names_set.groupby("APPS ID") if len(row) > 1)
print(len(duplicateAPPSid))
#duplicateAPPSid_duplBirths = pd.concat(row for _, row in Names_set.groupby("Date of Birth") if len(row) > 1)
#print(len(duplicateAPPSid_duplBirths))

#duplicateAPPSid.head()

Number of duplicate rows deleted: 0
54


In [36]:
duplicateAPPSid["FullName"] = duplicateAPPSid["Employee Name"] + ' ' + duplicateAPPSid["Family Name"] # Set new column of joined name parts
duplicateAPPSid["FullName"]=duplicateAPPSid["FullName"].str.replace('.','')
duplicateAPPSid["FullName"]=duplicateAPPSid["FullName"].str.strip()

In [37]:
duplicateAPPSid=duplicateAPPSid.sort_values(['APPS ID'], ascending=True)
duplicateIDs = list(set([i for i in duplicateAPPSid['APPS ID']]))

In [38]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
sameIdBirthName = {}
sameIdBirth = {}
for i in duplicateIDs:
    valuesList=[]
    df=duplicateAPPSid.loc[duplicateAPPSid['APPS ID'] == i]
    
    if len(df[df.duplicated(['FullName'])])>0:
        alikes=[]
        df_name_match = pd.concat(row for _, row in df.groupby('FullName') if len(row) > 1)
        for r in range(len(df_name_match)):
            alikes.append(df_name_match['index'].iloc[r])
        alikes=sorted(alikes)
        sameIdBirthName[alikes[0]]=alikes[1:]
        
    elif len(df[df.duplicated(['Date of Birth'])])>0:
        df_birthdays = pd.concat(row for _, row in df.groupby('Date of Birth') if len(row) > 1)
        for r in range(len(df_birthdays)):
            valuesList.append((df_birthdays['FullName'].iloc[r].lower().strip(),df_birthdays['index'].iloc[r]))
        alikes2 = []
        almostalikes=[]
        for idx,v in enumerate(valuesList):
            if v[0] in term_similarity_table.keys():
                if len(intersection([n[0] for n in term_similarity_table[v[0]]],[t[0] for t in valuesList]))>0:
                    alikes2.append(v[1])
                else:
                    almostalikes.append(v[1])
        if alikes2!=[]:
            alikes2 = sorted(alikes2)
            sameIdBirthName[alikes2[0]]=alikes2[1:]
        if almostalikes != []:
            almostalikes = sorted(intersection(almostalikes,[t[1] for t in valuesList]))
            sameIdBirth[almostalikes[0]]=almostalikes[1:]
   

In [39]:
sameIdBirthNameDF = pd.DataFrame(columns=Names_set.columns)
for idx,val in sameIdBirthName.items():
    cnt=1
    for rowidx in val:
        newrow = 'Alternative Entry' + str(cnt)
        #cond = Names_set.index = rowidx
        rows = Names_set.loc[Names_set.index == rowidx, :]
        sameIdBirthNameDF = sameIdBirthNameDF.append(rows, ignore_index=False)
        sameIdBirthNameDF = sameIdBirthNameDF.replace(np.nan, '', regex=True)
        Names_set.loc[idx, newrow] = str([ele for ele in sameIdBirthNameDF.loc[rowidx, :].values.tolist() if ele!=''])
        Names_set.drop(rows.index, inplace=True)
        cnt+=1

In [40]:
#sameIdBirthNameDF
#sameIdBirthNameDF['index'].iloc[rowidx]=idx

In [41]:
Names_set = Names_set.replace(np.nan, '', regex=True)
#Names_set

In [42]:
# writing to Excel
dataframe2Excel = pd.ExcelWriter('duplicates_apps_Cleaned.xlsx')
  
# write DataFrame to excel
Names_set.to_excel(dataframe2Excel)
  
# save the excel
dataframe2Excel.save()
dataframe2Excel.close()

## LEFT OFF HERE 7/16/2021

In [ ]:
AlternativesDF= sameIdBirthNameDF.reindex(list(range(0,sameIdBirthNameDF.index.max()+1)),fill_value='')
Cleaned_Names_set = Names_set.reindex(list(range(0,Names_set.index.max()+1)),fill_value='')

In [ ]:
for i in range(len(sameIdBirthNameDF)):
    rowindex=sameIdBirthNameDF['index'].iloc[i]
    Names_set.loc[rowIndex, 'Alternative Entry'] = list(row)
    print(rowindex)

In [ ]:
Names_set.iloc[14]

In [ ]:
Cleaned_Names_set